In [2]:
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
from scipy.fftpack import * 

前处理1：进行Ke谱分析的前处理的部分，考虑进行垂向的插值，计算垂直方向的平均值

In [11]:
model_type = "RCM"
# ke
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/ke_daily_vi/"
ds_ke    = xr.open_mfdataset(dir_in + "*_"+model_type+"_ke_daily_vi.nc") # 自动合并 Time 坐标
# zgrid
dir_in = "/raid52/yycheng/MPAS/"+model_type+"_postprocess/"+model_type+"_merge/hgt_daily_vi/"
ds_zgrid    = xr.open_mfdataset(dir_in + "*_"+model_type+"_hgt_daily_vi.nc") # 自动合并 Time 坐标
# ----- select data range -----
lat_sel     = (ds_ke.latitude >= 10) & (ds_ke.latitude <= 60)
lon_sel     = (ds_ke.longitude >= 70) & (ds_ke.longitude <= 140)
plevels_sel = (ds_ke.plevels >= 100) & (ds_ke.plevels <= 400)
# plevels_sel = (ds_ke.plevels == 100)
time_sel    = (ds_ke.Time.dt.year >= 1998)
sel_dict    = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":time_sel}

In [ ]:
# 进行垂直积分，求垂直平均值
ke_sel = ds_vr_ke.isel(sel_dict)        # 读入选取数据到 ke_sel
sel_plevels_val = ke_sel.plevels.values # 转换为np.array
ke_sum = np.trapz(ke_sel['ke'].values, sel_plevels_val, axis = 3) / (400. - 100.)
# 154 x 160 x 279 3d np array [time,lat,lon]

In [ ]:
# np.isnan(vr_ke_sum).any() # 检查，显然是有NAN的
# 将numpy的nan处理为单独的数 -1e30，方便后续写入文件之中
# 将RCM的边缘NAN和本身插值造成的NAN一并处理掉了
ke_sum_rmnan = np.where(ke_sum>1e10, -1e30, vr_ke_sum)

In [ ]:
# 创建临时输出，方便后面再进行计算
dir_out = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/ke_daily_vi/"
# list(vr_ke_sel.coords)
coords_lon  = ke_sel.longitude
coords_lat  = ke_sel.latitude
coords_Time = ke_sel.Time
# xr.DataArray(vr_ke_sum,dims = ['longitude', 'latitude', 'Time'])
temp_write = xr.DataArray(ke_sum_rmnan, name = "vertical integration Ke", coords = [coords_Time, coords_lat ,coords_lon],attrs={"caculation":"100-400hPa Ke trapz integration ( / 300hPa)"})
# temp_write.values = vr_ke_sum # 直接向values中写入
encoding_dict = {"vertical integration Ke":{"_FillValue":-1e30}}
temp_write.to_netcdf(dir_out + "ke_all_400-100_hPa_vi.nc",'w',\
    encoding = encoding_dict) # 此处进行NAN的处理：修改encoding方法